# FIN Model

In [120]:
import numpy as np
from numpy import random
import pandas as pd
from scipy.signal import cwt,ricker
from scipy import stats
from sklearn.preprocessing import StandardScaler,normalize
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,concatenate,Layer,Dense,Activation,MaxPooling2D,Dropout,Conv2D,BatchNormalization,Reshape,UpSampling2D,ZeroPadding2D

In [121]:
random.seed(222)

# Task 1

Train two feature imitating networks that take as input a fixed length sequence and provide as output (1) the mean and (2) the standard deviation of a fixed length input; a deep feedforward architecture should be sufficient for this.

In [122]:
# Z-scored np array
# T x Ch x Epochs
N = 600 # Fixed number of length data
E = 150 # Number of Epochs
input_arr = np.random.rand(N,1,E)
scaled_arr = np.zeros(input_arr.shape)
for ii in range(input_arr.shape[1]):
    scaler = StandardScaler()
    scaled_arr[:,ii,:] = scaler.fit_transform(input_arr[:,ii,:])
    
#print(scaled_arr.shape)

# move the axis of the matrix as as # Epochs X T X Ch

scaled_arr = np.moveaxis(scaled_arr,2,0)
print(scaled_arr.shape)

do_cwt = True
if do_cwt:
    # Continuous wavelet transform
    flat_data = np.squeeze(scaled_arr)
    scaled_arr_cwt = []
    for ii in range(flat_data.shape[0]):
        widths = np.arange(1, 31)
        cwtmatr = cwt(flat_data[ii], ricker, widths)
        scaled_arr_cwt.append(np.moveaxis(cwtmatr,1,0))
    scaled_arr3d = np.stack(scaled_arr_cwt)
else:
    scaled_arr3d = scaled_arr
    
print(scaled_arr3d.shape)

(150, 600, 1)
(150, 600, 30)


In [123]:
# Will use kurtosis for training, so calculate the kurtosis of the data
def Kurtosis(eegData):
    return stats.kurtosis(scaled_arr,axis=1)

#squeeze the data in
feature_res = np.squeeze(Kurtosis(scaled_arr))
#print(feature_res)

In [124]:
# Make FIN
def make_FIN(topology,size3D,N,name="my-FIN"):
    input_layer = keras.Input(shape=(N,size3D))
    x = Flatten()(input_layer)
    for layer in topology['layers']:
        x = Dense(layer['output_shape'], activation=layer['activation'])(x)
        if layer['BatchNorm']:
            x = BatchNormalization(axis=1)(x)
    fin_out = Dense(1, activation='linear')(x)
    fin = keras.Model(input_layer, fin_out, name=name)
    return fin

topology = np.load('RandomTopology.npy', allow_pickle=True)[1]


fin = make_FIN(topology,30,N)


In [126]:
learning_rate=topology['lr']
momentum=topology['momentum']
nesterov=topology['nesterov']
b1 = topology['b1']
b2 = topology['b2']
if topology['opt'] == 'SGD':
    grad_desc_algorithm = tf.keras.optimizers.SGD(learning_rate=learning_rate,momentum=momentum,nesterov=nesterov)
if topology['opt'] == 'ADAM':
    grad_desc_algorithm = tf.keras.optimizers.Adam(learning_rate=learning_rate,beta_1=b1,beta_2=b2)
    
fin.compile(optimizer = grad_desc_algorithm,loss = 'categorical_crossentropy')

loss_d = fin.fit(scaled_arr3d, feature_res, 
            epochs = 8,
            batch_size = N,
            verbose   = 1)            

Epoch 1/8
1/1 [==============================] - 1s 703ms/step - loss: -1.4276e-07
Epoch 2/8
1/1 [==============================] - 0s 75ms/step - loss: -1.4276e-07
Epoch 3/8
1/1 [==============================] - 0s 50ms/step - loss: -1.4276e-07
Epoch 4/8
1/1 [==============================] - 0s 67ms/step - loss: -1.4276e-07
Epoch 5/8
1/1 [==============================] - 0s 38ms/step - loss: -1.4276e-07
Epoch 6/8
1/1 [==============================] - 0s 61ms/step - loss: -1.4276e-07
Epoch 7/8
1/1 [==============================] - 0s 67ms/step - loss: -1.4276e-07
Epoch 8/8
1/1 [==============================] - 0s 53ms/step - loss: -1.4276e-07
